# Perceptron implementation

In yesterday's lecture you saw perceptron models, a type of linear classifier that does not depend on a probability distribution. In today's practicum we'll walk through a simple Python implementation. We'll begin with a binary classifier, generalize it to multinomial classification, and finally add weight averaging.

## Binary classification

Recall from the lecture that a binary perceptron's parameters consist of a bias term $b$ and a weight vector $\beta$, both real-valued. For individual parameters, we define a simple class which has a weight and an `update` method.

In [6]:
from typing import Union


Num = Union[int, float]


class Weight:
    """Simple perceptron weight."""
    
    def __init__(self, value: Num = 0):
        self.w = value
    
    def __repr__(self):
        return f"current state {self.__class__.__name__}({self.w})"
    
    def update(self, tau: int) -> None:
        self.w += tau
    
    # We'll allow this to be integral- or real-valued.
    def value(self) -> Num:
        return self.w

Let's play with it a bit.

In [8]:
w = Weight(0)
print(w)
w.update(+1)
w.update(-1)
w.update(-1)
w.update(-1)
print(w)

current state Weight(0)
current state Weight(-2)


For simplicity, we assume that features are strings, and define $\beta$ as a dictionary mapping from feature strings to `Weights`. We also define two methods:

* `predict` will compute $\sigma$ then apply the decision function.
* `update` will apply the perceptron update rule.

In [9]:
import collections

from typing import List


FeatureVector = List[str]


class BinaryPerceptron:
    """Simple binary perceptron, part 1."""
    
    def __init__(self):
        self.b = Weight()
        self.beta = collections.defaultdict(Weight)
    
    def predict(self, features: FeatureVector) -> bool:
        """Returns the prediction."""
        # Computes sigma.
        score  = self.b.value()
        for feature in features:
            score += self.beta[feature].value()
        # Applies the decision function.
        return score >= 0
    
    def update(self, features: FeatureVector, y: bool):
        """Applies the update rule."""
        yhat = self.predict(features)
        tau = y - yhat
        if not tau:
            return
        # Actually updates.
        self.b.update(tau)
        for feature in features:
            self.beta[feature].update(tau)

Now, let's use this to classify. We'll reuse the pet names example from two weeks ago.

In [10]:
import random

import nltk

# I had to install the corpus ahead of time. But you only have to do this once.
assert nltk.download("names")

female = nltk.corpus.names.words("female.txt")
print(f"Loaded {len(female)} female names")
male = nltk.corpus.names.words("male.txt")
print(f"Loaded {len(male)} male names")

# Then, we'll concatenate the data and turn it into (x, y) pairs.
name_list = [(name, "female") for name in female] + [(name, "male") for name in male]

# Then, we'll shuffle the data.
random.seed(562)
random.shuffle(name_list)  # This works in place.

# Then, we'll split it into training and test data.
train = name_list[:-1000]
test = name_list[-1000:]

[nltk_data] Downloading package names to /Users/hmghaly/nltk_data...
[nltk_data]   Package names is already up-to-date!
Loaded 5001 female names
Loaded 2943 male names


In [5]:
import string


# I have modified the feature extraction code to return strings.


FeatureVector = List[str]


def extract_features(name: str) -> FeatureVector:
    """Extracts features for a single example."""
    name_lowercase = name.casefold()
    vowels = frozenset("aeiou")
    features = []
    if name[0] in vowels:
        features.append("startswith(vowel)")
    if name[-1] in vowels:
        features.append("endswith(vowel)")
    for char in string.ascii_lowercase:
        count = name.count(char)
        if count:
            features.append(f"count({char})={count}")
            features.append(f"has({char})")
    return features


# An example feature vector.
print(extract_features("Bodie"))

['endswith(vowel)', 'count(d)=1', 'has(d)', 'count(e)=1', 'has(e)', 'count(i)=1', 'has(i)', 'count(o)=1', 'has(o)']


In [6]:
train_set = [(extract_features(name), label == "female") for (name, label) in train]

# An example pair.
print(train_set[0])

(['count(a)=1', 'has(a)', 'count(d)=1', 'has(d)', 'count(e)=2', 'has(e)', 'count(l)=1', 'has(l)', 'count(n)=1', 'has(n)', 'count(r)=1', 'has(r)', 'count(x)=1', 'has(x)'], False)


Now we are ready to train. We'll do 5 epochs.

In [8]:
model = BinaryPerceptron()

# For each of 5 epochs...
for i in range(5):
    # For each example in that epoch...
    for (features, y) in train_set:
        model.update(features, y)

Now, let's evaluate and compute accuracy (the percentage of correctly classified test examples).

In [9]:
test_set = [(extract_features(name), label == "female") for (name, label) in test]

examples = len(test_set)
correct = 0
for (features, y) in test_set:
    if y == model.predict(features):
        correct +=1
        
print(f"Accuracy:\t{correct / examples:.4f}")

Accuracy:	0.7270


Not bad.

### Stretch goals

* Add a `fit` method to `BinaryPerceptron` that automates the training loops. It should take a list of label/feature pairs and the number of epochs.
* Write a function (not a method of `BinaryPerceptron`) to determine accuracy.
* Determine accuracy (percentage of correct classifications) as a function of the number of epochs. After how many epochs does performance start declining?

## Multinomial perceptron

The generalization to the multinomial perceptron is straightforward. Recall from the lecture that the parameters consist of a bias vector $B$ and a weight matrix $\beta$, both real-valued. We will assume here that the labels themselves are also strings. Furthermore, whereas in the lecture we wrote a particular weight as $\beta_{i,y'}$, here we will treat $y'$ as the outer key and $i$ as the inner key. (You'll see what I mean in a second).

In [ ]:
import functools
import operator


class MultinomialPerceptron:
    
    def __init__(self):
        self.b = collections.defaultdict(Weight)
        # A nested defaultdict looks a bit like this...
        self.beta = collections.defaultdict(
            functools.partial(
                collections.defaultdict,
                Weight
            )
        )
        
      
    def predict(self, features: FeatureVector) -> str:
        """Computes the score for a feature bundle."""
        # Computes sigma.
        scores = {yprime: weight.value() for (yprime, weight)
                  in self.b.items()}
        # At the beginning, it won't know about the `Y` set,
        # but we have to return something.
        if not scores:
            return ""
        for feature in features:
            for (yprime, weight) in self.beta[feature].items():
                print(yprime, weight)
                scores[yprime][feature] += weight.value()
        # Computes yhat using argmax on a dictionary.
        return max(scores.items(), key=operator.itemgetter(1))
    
    def update(self, features: FeatureVector, y: str) -> None:
        """Applies the update rule."""
        yhat = self.predict(features)
        if y == yhat:
            return
        # Actually updates.
        self.b[y].update(+1)
        self.b[yhat].update(-1)
        for feature in features:
            self.beta[y][feature].update(+1)
            self.beta[yhat][feature].update(-1)

### Stretch goals

* Apply this to some interesting linguistic problem. Some debugging may be required!

## Weight averaging

Finally, we can add weight averaging. One way to do this is to define two variants of `BinaryPerceptron` or `MultinomialPerceptron` each:

* one which uses averaging weights instead of `Weight`
* one which uses normal `Weight`s and is created by applying averaging

The following is the averaging weight that described in class, except that:
    
* I have added a `__repr__` method.
* I have added a method `value` to return the current weight (the one used during training).
* I have modified `average` so that it returns a `Weight` instance.

In [10]:
class LazyWeight:
    """Lazily averaged weight."""

    def __init__(self):
        self.w_c = 0
        self.t = 0
        self.sum_w = 0
        
    def __repr__(self):
        return (f"{self.__class__.__name__}"
                f"(w_c={self.w_c}, t={self.t})")
    
    def freshen(self, c: int) -> None:
        self.sum_w += (c - self.t) * self.w_c
        self.t = c
        
    def update(self, tau: int, c: int) -> None:
        self.freshen(c)
        self.w_c += tau
    
    def value(self, c: int) -> int:
        return self.w_c
        
    def average(self, c: int) -> Weight:
        self.freshen(c)
        return Weight(self.sum_w / c)

Let's play with this a little.

In [11]:
c = 0
w = LazyWeight()
# Some time passes...
c += 100
w.update(+1, c)
# Some more time passes.
c += 150
w.update(-1, c)
print(w)
# Now we're done.
print(w.average(c))

LazyWeight(w_c=0, t=250)
Weight(0.6)


### Stretch goals

* Build `BinaryAveragingPerceptron` which uses `LazyWeight` (instead of `Weight`) parameters.
* Then, define a method or function which populates a `BinaryPerceptron` with the result of averaging these weights.
* Then repeat the evaluations above.

# Further reading

For a complete Python implementation of a variety of perceptron classifiers, see [`nlup.perceptron`](https://github.com/cslu-nlp/nlup/blob/master/nlup/perceptron.py).